In [ ]:
# A_Aug_ ... Add file name
import os
import cv2
import numpy as np

def create_merged_filename(file_name, merge_from_job, source_class_id, merge_to_job, target_class, extension):
    """
    Create a new filename for the merged image or annotation based on user specifications.
    """
    base_name, _ = os.path.splitext(file_name)  # Retain the original file name
    new_name = (f"Merge_RY_{merge_from_job}_into_{merge_to_job}_class_{target_class}_{base_name}{extension}")
    return new_name

def check_files_exist(src_image_path, tgt_image_path, src_annotation_path, tgt_annotation_path):
    """
    Check if all required files exist and return True or False.
    """
    if not os.path.exists(src_image_path):
        print(f"Source image not found: {src_image_path}. Skipping.")
        return False
    if not os.path.exists(tgt_image_path):
        print(f"Target image not found: {tgt_image_path}. Skipping.")
        return False
    if not os.path.exists(src_annotation_path):
        print(f"Source annotation not found: {src_annotation_path}. Skipping.")
        return False
    if not os.path.exists(tgt_annotation_path):
        print(f"Target annotation not found: {tgt_annotation_path}. Skipping.")
        return False
    return True

def fill_bounding_box_with_background(tgt_image, annotation, target_class):
    """
    Fill the bounding box for the target class with a sampled background color.
    """
    image_height, image_width = tgt_image.shape[:2]
    target_bbox = None

    for class_id, x_center, y_center, width, height in annotation:
        if int(class_id) == target_class:
            x_min = int((x_center - width / 2) * image_width)
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width)
            y_max = int((y_center + height / 2) * image_height)

            # Sample background color from the top row of the bounding box
            top_row = tgt_image[y_min, x_min:x_max]
            background_color = top_row.mean(axis=0).astype(np.uint8)

            # Fill the bounding box with the sampled background color
            tgt_image[y_min:y_max, x_min:x_max] = background_color
            target_bbox = (x_min, y_min, x_max, y_max)
            break

    if target_bbox is None:
        print("No bounding box found for target class. Skipping background filling.")
    return tgt_image, target_bbox

def overlay_mask_on_bbox(src_mask, tgt_image, bbox_coords, scale_factor=1.0, x_offset=0):
    """
    Overlay the source mask on the bounding box area with a horizontal offset and scale factor.
    """
    x_min, y_min, x_max, y_max = bbox_coords

    # Resize mask to match bounding box dimensions with scale factor
    tgt_width = int((x_max - x_min) * scale_factor)
    tgt_height = int((y_max - y_min) * scale_factor)
    resized_mask = cv2.resize(src_mask, (tgt_width, tgt_height), interpolation=cv2.INTER_LINEAR)

    # Apply the x-offset
    x_min_offset = max(0, x_min + x_offset)
    x_max_offset = min(tgt_image.shape[1], x_min_offset + tgt_width)

    # Place the resized mask in the target image
    for c in range(3):
        tgt_image[y_min:y_min+tgt_height, x_min_offset:x_max_offset, c] = \
            resized_mask[:, :, c] * (resized_mask[:, :, 3] > 0) + \
            tgt_image[y_min:y_min+tgt_height, x_min_offset:x_max_offset, c] * (resized_mask[:, :, 3] == 0)

    return tgt_image, (x_min_offset, y_min, x_min_offset + tgt_width, y_min + tgt_height)

def process_images_with_mask_overlay(src_folder, tgt_folder, merge_folder, target_class, class_to_take, scale_factor=1.0, x_offset=0):
    """
    Process images and overlay masks with annotations and horizontal offset.
    """
    if not os.path.exists(merge_folder):
        os.makedirs(merge_folder)

    for file_name in os.listdir(src_folder):
        if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        src_image_path = os.path.join(src_folder, file_name)
        tgt_image_path = os.path.join(tgt_folder, file_name)
        src_annotation_path = os.path.join(src_folder, os.path.splitext(file_name)[0] + ".txt")
        tgt_annotation_path = os.path.join(tgt_folder, os.path.splitext(file_name)[0] + ".txt")

        # Check file existence
        if not check_files_exist(src_image_path, tgt_image_path, src_annotation_path, tgt_annotation_path):
            continue

        # Load images and annotations
        src_image = cv2.imread(src_image_path)
        tgt_image = cv2.imread(tgt_image_path)
        with open(src_annotation_path, 'r') as f:
            src_annotations = [list(map(float, line.split())) for line in f]
        with open(tgt_annotation_path, 'r') as f:
            tgt_annotations = [list(map(float, line.split())) for line in f]

        # Create mask from the source image within the bounding box of the source class
        mask_path = os.path.join(merge_folder, f"mask_{file_name}")
        src_mask, _ = create_mask_within_bbox(src_image, src_annotations, class_to_take, mask_path)

        # Fill the bounding box for the target class with background color
        tgt_image, bbox_coords = fill_bounding_box_with_background(tgt_image, tgt_annotations, target_class)
        if bbox_coords is None:
            print(f"No bounding box for target class in {file_name}. Skipping.")
            continue

        # Overlay mask onto target image
        updated_image, new_bbox = overlay_mask_on_bbox(src_mask, tgt_image, bbox_coords, scale_factor, x_offset)

        # Update annotations
        updated_annotation = update_annotations_with_mask(new_bbox, tgt_image.shape, class_to_take)

        # Save output image and annotations
        new_image_filename = create_merged_filename(file_name, '30', class_to_take, '24', target_class, '.png')
        new_annotation_filename = create_merged_filename(file_name, '30', class_to_take, '24', target_class, '.txt')

        output_image_path = os.path.join(merge_folder, new_image_filename)
        output_annotation_path = os.path.join(merge_folder, new_annotation_filename)

        cv2.imwrite(output_image_path, updated_image)
        save_updated_annotations(tgt_annotations, updated_annotation, target_class, output_annotation_path)
        print(f"Processed: {output_image_path}, Annotation: {output_annotation_path}")

# ------------------------ Parameters ------------------------
src_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_out_of_30"
tgt_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_into_24_Filter"
merge_folder = "D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24"
class_to_take = 1  # Source class ID
target_class = 0   # Target class ID
scale_factor = 1.1  # Scale factor for resizing mask_RY_
x_offset = -15      # Horizontal offset for the overlay

process_images_with_mask_overlay(src_folder, tgt_folder, merge_folder, target_class, class_to_take, scale_factor, x_offset)

# ------------------------ Parameters ------------------------
#src_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_out_of_30"
#tgt_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_into_24_Filter"
#merge_folder = "D:/FlagDetectionDatasets/Augmentation/Merged_30_to_24"
#class_to_take = 1  # Source class ID
#target_class = 0   # Target class ID
#scale_factor = 1.2  # Scale factor for resizing mask

#process_images_with_mask_overlay(src_folder, tgt_folder, merge_folder, target_class, class_to_take, scale_factor)

# ------------------------- Parameters -------------------------
#src_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_out_of_30"
#tgt_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_into_24_F2"
#merge_folder = "D:/FlagDetectionDatasets/Augmentation/Merged_RY7_30_to_24"
#class_to_take = 1  # Source class
#target_class = 0   # Target class
#scale_factor = 1.2  # Scale factor for mask

process_images_with_mask_overlay(src_folder, tgt_folder, merge_folder, target_class, class_to_take, scale_factor)

# ------------------------- Parameters -------------------------
#src_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_out_of_30"
#tgt_folder = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_into_24_Filter"
#merge_folder = "D:/FlagDetectionDatasets/Augmentation/MergedRY4_30_to_24"
#class_to_take = 1  # Source class
#target_class = 0   # Target class
#scale_factor = 1.2  # my defined scale factor

# Paths and parameters
#merge_class_from = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_out_of_30"
#merge_class_into = "D:/FlagDetectionDatasets/Augmentation/Switch_flag_into_24_Filter"
#merge_fldr = "D:/FlagDetectionDatasets/Augmentation/Merged_RY2_30_to_24"
##class_to_take = 1  # Source class (red/yellow flag)
#class_to_replace = 0  # Target class (red flag)
#merge_from_job = '30'
#merge_to_job = '24'



Intermediate mask saved to: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\mask_000173.PNG
Annotations saved to: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\Merge_RY_30_into_24_class_0_000173.txt
Processed: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\Merge_RY_30_into_24_class_0_000173.png, Annotation: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\Merge_RY_30_into_24_class_0_000173.txt
Intermediate mask saved to: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\mask_000174.PNG
Annotations saved to: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\Merge_RY_30_into_24_class_0_000174.txt
Processed: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\Merge_RY_30_into_24_class_0_000174.png, Annotation: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\Merge_RY_30_into_24_class_0_000174.txt
Intermediate mask saved to: D:/FlagDetectionDatasets/Augmentation/MergeRY1_30_To_24\mask_000175.PNG
Annotations saved to: D:/FlagDetection